In [1]:
#!/usr/bin/env python
# coding: utf-8

from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
intermediate_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Intermediates.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
robot.reset()

def stock_solution (intermediate, solvent):
    
    #Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    destination_int = containers.load("FluidX_24_5ml", "A1", "int")
    trash = containers.load("point", "C3")
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    id_header = "CPD ID"    
    solvent = "MeCN"
    stock_sol1 = "Intermediate1"
    stock_sol2 = "Intermediate2"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (stock)"
    volume_per_vial = "Volume to dispense"
    stock1 = "ImidInt-1"
    stock2 = "ImidInt-2"
    code_header = "Code"
    
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
        if x == stock_sol1:
            stock_sol1_loc = solvent_df[location_header].tolist()[i]
            stock_sol1_volume = solvent_df[volume_stock_header].tolist()[i]
        if x == stock_sol2:
            stock_sol2_loc = solvent_df[location_header].tolist()[i]
            stock_sol2_volume = solvent_df[volume_stock_header].tolist()[i]
    p1000.pick_up_tip()
    p1000.transfer([stock_sol1_volume], source_trough4row.wells(solvent_location), source_trough4row.wells(stock_sol1_loc).top(-5), new_tip = 'never')
    p1000.drop_tip()
    p1000.pick_up_tip()
    p1000.transfer([stock_sol2_volume], source_trough4row.wells(solvent_location), source_trough4row.wells(stock_sol2_loc).top(-5), new_tip = 'never')
    p1000.drop_tip()
    print(solvent_location, stock_sol1_loc, stock_sol2_loc, stock_sol1_volume, stock_sol2_volume)

    robot.pause()
    for i, x in enumerate(intermediate_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [intermediate_df[volume_per_vial].tolist()[i]]
        intermediate_id = intermediate_df[code_header].tolist()[i]
        if intermediate_id == stock1:
            print ('correct')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol1_loc), destination_int.wells(destination_location).top(-5), new_tip = 'never')   
        if intermediate_id == stock2:
            print ('incorrect')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol2_loc), destination_int.wells(destination_location).top(-5), new_tip = 'never')   
        #else:
         #   print ("error")
          #  break

        print(destination_location,vol_to_dispense,intermediate_id, stock1)
    
stock_solution(intermediate_df, solvent_df)
#robot.commands()


KeyError: 'Volume to dispense (stock)'

In [5]:
#!/usr/bin/env python
# coding: utf-8

# In[22]:


from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
reaction_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\reaction_step1_MeI.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
robot.reset()

def intermediate_transfer (reaction, solvent):
    #Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough12row = containers.load("trough-12row", "C1")
    location_int = containers.load("FluidX_24_5ml", "A1", "int")
    reaction_rack = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    location_header = "Location"
    #stock1 = "ImidInt-1"
    #stock2 = "ImidInt-2"
    #code_header = "Code"
    nb_reaction_header = "number of reaction"
    volume_per_reaction_header = "Volume_stock per reaction"
    
    n=0
    for i, x in enumerate(reaction_df[location_header].tolist()):
        
        nb_reaction = int(reaction_df[nb_reaction_header].tolist()[i])
        volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[i])
        source_location = reaction_df[location_header].tolist()[i]
        volume_list = []
        destination_list = []
        #for i in range (n+0, n+nb_reaction):
            #destination_list.append(i)
            #volume_list.append(volume_per_reaction)
        p1000.distribute(volume_per_reaction, location_int.wells(source_location), [x.top() for x in reaction_rack.wells(n+0, to=n + nb_reaction-1)])
        n = nb_reaction+n

intermediate_transfer(reaction_df, solvent_df)

robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 990.0 at <Deck><Slot A1><Container int><Well A1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well E1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well F1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well G1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well H1>',
 'Blowing out at <Deck><Slot C3><Container point><Well A1>',
 'Aspirating 990.0 at <Deck><Slot A1><Container int><Well A1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A2>',
 'Dispensing 120.0 at <Deck><Slot D1><Container St

In [6]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
amines_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\amines_1_template.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
#robot.reset()


def stock_solution_amines(amines, solvent):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_AM_1 = containers.load("FluidX_24_5ml", "A1", "AM_1")
    rack_stock_AM_2 = containers.load("FluidX_24_5ml", "A2", "AM_2")
    rack_stock_AM_3 = containers.load("FluidX_24_5ml", "A3", "AM_3")
    rack_stock_AM_4 = containers.load("FluidX_24_5ml", "B1", "AM_4")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMF"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.5M"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
    # robot.pause()
    for i, x in enumerate(amines_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [amines_df[volume_stock_header].tolist()[i]]
        amine_id = amines_df[rack_ID_header].tolist()[i]
        if amine_id == "":
            print ('null')
            break
        print (rack_ID_header, amine_id, vol_to_dispense)
        p1000.pick_up_tip()
        if amine_id == rack_1:
            print ('rack1')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_1.wells(destination_location).top(-5), new_tip='never')
        if amine_id == rack_2:
            print ('rack2')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_2.wells(destination_location).top(-5), new_tip='never')
        if amine_id == rack_3:
            print ('rack3')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_3.wells(destination_location).top(-5), new_tip='never')
        if amine_id == rack_4:
            print ('rack4')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_4.wells(destination_location).top(-5), new_tip='never')
        p1000.drop_tip()
stock_solution_amines(amines_df, solvent_df)

Rack ID 24_rack1 ['1443.742099']
rack1
Rack ID 24_rack1 ['1337.914743']
rack1
Rack ID 24_rack1 ['1536.755067']
rack1
Rack ID 24_rack1 ['1620.708196']
rack1
Rack ID 24_rack1 ['1804.132662']
rack1
Rack ID 24_rack1 ['1635.369622']
rack1
Rack ID 24_rack1 ['1787.585527']
rack1
Rack ID 24_rack1 ['1850.95455']
rack1
Rack ID 24_rack1 ['1841.054313']
rack1
Rack ID 24_rack1 ['1475.169751']
rack1
Rack ID 24_rack1 ['1811.909949']
rack1
Rack ID 24_rack1 ['1881.541389']
rack1
Rack ID 24_rack1 ['1966.356863']
rack1
Rack ID 24_rack1 ['1782.54783']
rack1
Rack ID 24_rack1 ['1903.199344']
rack1
Rack ID 24_rack1 ['1818.013686']
rack1
Rack ID 24_rack1 ['1944.074567']
rack1
Rack ID 24_rack1 ['1873.184555']
rack1
Rack ID 24_rack1 ['1837.999082']
rack1
Rack ID 24_rack1 ['1889.07112']
rack1
Rack ID 24_rack1 ['1794.401101']
rack1
Rack ID 24_rack1 ['1894.359587']
rack1
Rack ID 24_rack1 ['0.066']
rack1
Rack ID 24_rack1 ['1529.352055']
rack1
Rack ID 24_rack2 ['1940.722696']
rack2
Rack ID 24_rack2 ['1823.919728']
r

RuntimeWarning: eppendorf1000 has run out of tips

In [8]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
amines_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\drive-download-20190220T090113Z-001\amines_1_template (1).csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\drive-download-20190220T090113Z-001\Solvents.csv")
robot.reset()


def stock_solution_amines(amines, solvent):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_AM_1 = containers.load("FluidX_24_5ml", "A1", "AM_1")
    rack_stock_AM_2 = containers.load("FluidX_24_5ml", "A2", "AM_2")
    rack_stock_AM_3 = containers.load("FluidX_24_5ml", "A3", "AM_3")
    rack_stock_AM_4 = containers.load("FluidX_24_5ml", "B1", "AM_4")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMF"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.5M"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
    # robot.pause()
    for i, x in enumerate(amines_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [amines_df[volume_stock_header].tolist()[i]]
        amine_id = amines_df[rack_ID_header].tolist()[i]
        if amine_id == "":
            print ('null')
            break
        print (rack_ID_header, amine_id, vol_to_dispense)
        #p1000.pick_up_tip()
        if amine_id == rack_1:
            print ('rack1')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_1.wells(destination_location).top(-5), new_tip='never')
        if amine_id == rack_2:
            print ('rack2')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_2.wells(destination_location).top(-5), new_tip='never')
        if amine_id == rack_3:
            print ('rack3')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_3.wells(destination_location).top(-5), new_tip='never')
        if amine_id == rack_4:
            print ('rack4')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_AM_4.wells(destination_location).top(-5), new_tip='never')
        #from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
amines_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\amines_1_template_corrected.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
#robot.reset()

def amine_transfer (amines, solvent):
    #Deck setup
    
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "B2")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_AM_1 = containers.load("FluidX_24_5ml", "A1", "AM_1")
    rack_stock_AM_2 = containers.load("FluidX_24_5ml", "A2", "AM_2")
    rack_stock_AM_3 = containers.load("FluidX_24_5ml", "A3", "AM_3")
    rack_stock_AM_4 = containers.load("FluidX_24_5ml", "B1", "AM_4")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    reaction_rack_2 = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000,tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"    
    solvent = "DMF"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent = "DMF"
    solvent_volume_header = "Volume to dispense (uL)"
    counter =0
    for i, x in enumerate(amines_df[rack_ID_header].tolist()):
        volume_per_reaction = [amines_df[volume_per_reaction_header].tolist()[i]]
        source_location = amines_df[location_header].tolist()[i]
        p1000.pick_up_tip()
        if x == "":
            print ('null')
            break
        if x == rack_1:
            #p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack.wells(i+2).top(-5))
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_2:
            #p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_2.wells(source_location), reaction_rack.wells(i+2).top(-5))
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_3:
            #p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_3.wells(source_location), reaction_rack.wells(i+2).top(-5))
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_4:
            #p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_4.wells(source_location), reaction_rack.wells(i+2).top(-5))
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        p1000.drop_tip()
        counter+=1
    p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]
    
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc), [x.top() for x in reaction_rack.wells(0, to=counter-1)])
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),[x.top() for x in reaction_rack_2.wells(0, to=counter - 1)])
    p1000.drop_tip()
    #print (counter)
    robot.home()
amine_transfer(amines_df, solvent_df)p1000.drop_tip()
stock_solution_amines(amines_df, solvent_df)

robot.commands()

Rack ID 24_rack1 ['1443.742099']
rack1
Rack ID 24_rack1 ['1337.914743']
rack1
Rack ID 24_rack1 ['1536.755067']
rack1
Rack ID 24_rack1 ['1620.708196']
rack1
Rack ID 24_rack1 ['1804.132662']
rack1
Rack ID 24_rack1 ['1635.369622']
rack1
Rack ID 24_rack1 ['1787.585527']
rack1
Rack ID 24_rack1 ['1850.95455']
rack1
Rack ID 24_rack1 ['1841.054313']
rack1
Rack ID 24_rack1 ['1475.169751']
rack1
Rack ID 24_rack1 ['1811.909949']
rack1
Rack ID 24_rack1 ['1881.541389']
rack1
Rack ID 24_rack1 ['1966.356863']
rack1
Rack ID 24_rack1 ['1782.54783']
rack1
Rack ID 24_rack1 ['1903.199344']
rack1
Rack ID 24_rack1 ['1818.013686']
rack1
Rack ID 24_rack1 ['1944.074567']
rack1
Rack ID 24_rack1 ['1873.184555']
rack1
Rack ID 24_rack1 ['1837.999082']
rack1
Rack ID 24_rack1 ['1889.07112']
rack1
Rack ID 24_rack1 ['1794.401101']
rack1
Rack ID 24_rack1 ['1894.359587']
rack1
Rack ID 24_rack1 ['0.066']
rack1
Rack ID 24_rack1 ['1529.352055']
rack1
Rack ID 24_rack2 ['1940.722696']
rack2
Rack ID 24_rack2 ['1823.919728']
r

['Aspirating 721.8710495 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 721.8710495 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Aspirating 721.8710495 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 721.8710495 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Aspirating 668.9573715 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 668.9573715 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Aspirating 668.9573715 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 668.9573715 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Aspirating 768.3775335 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 768.3775335 at <Deck><Slot A1><Container AM_1><Well A3>',
 'Aspirating 768.3775335 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 768.3775335 at <Deck><Slot A1><Container AM_1><Well A3>',
 'Aspirating 810.354098 at <Deck><Slot C2><Container trough-12row><Well A4>',
 'Dispensing 810.35409

In [19]:
from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
amines_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\amines_1_template_corrected.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
robot.reset()

def amine_transfer (amines, solvent):
    #Deck setup
    
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "B2")
    tiprack_1000_3 = containers.load("tiprack-1000ul-H", "B1")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_AM_1 = containers.load("FluidX_24_5ml", "A1", "AM_1")
    rack_stock_AM_2 = containers.load("FluidX_24_5ml", "A2", "AM_2")
    rack_stock_AM_3 = containers.load("FluidX_24_5ml", "A3", "AM_3")
    rack_stock_AM_4 = containers.load("FluidX_24_5ml", "B1", "AM_4")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    reaction_rack_2 = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000,tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"    
    solvent = "DMF"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent = "DMF"
    solvent_volume_header = "Volume to dispense (uL)"
    counter =0
    for i, x in enumerate(amines_df[rack_ID_header].tolist()):
        volume_per_reaction = [amines_df[volume_per_reaction_header].tolist()[i]]
        source_location = amines_df[location_header].tolist()[i]
        
        if x == "":
            print ('null')
            break
        if x == rack_1:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_2:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_2.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_3:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_3.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_4:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_4.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        
        counter+=1
    #p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]
    
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc), [x.top() for x in reaction_rack.wells(0, to=counter + 1)])
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),[x.top() for x in reaction_rack_2.wells(0, to=counter + 1)])
    
    print (counter)
    robot.home()
amine_transfer(amines_df, solvent_df)
robot.commands()

null
58


['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well D1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A3>',
 'Dispensing 132.0 at <D

In [20]:
print('youre silly')

youre silly


In [22]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
amines_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\amines_1_template_corrected.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
#robot.reset()


def amine_transfer(amines, solvent):
    # Deck setup

    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "B2")
    tiprack_1000_3 = containers.load("tiprack-1000ul-H", "B1")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_AM_1 = containers.load("FluidX_24_5ml", "A1", "AM_1")
    rack_stock_AM_2 = containers.load("FluidX_24_5ml", "A2", "AM_2")
    rack_stock_AM_3 = containers.load("FluidX_24_5ml", "A3", "AM_3")
    rack_stock_AM_4 = containers.load("FluidX_24_5ml", "B1", "AM_4")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    reaction_rack_2 = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")

    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000, tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )

    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMF"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent = "DMF"
    solvent_volume_header = "Volume to dispense (uL)"
    counter = 0
    for i, x in enumerate(amines_df[rack_ID_header].tolist()):
        volume_per_reaction = [amines_df[volume_per_reaction_header].tolist()[i]]
        source_location = amines_df[location_header].tolist()[i]

        if x == "":
            #print('null')
            break
        if x == rack_1:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),
                           reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_2:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_2.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),
                           reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_3:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_3.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),
                           reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_4:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_4.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),
                           reaction_rack_2.wells(i + 2).top(-5))

        counter += 1
    # p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]

    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),
                     [x.top() for x in reaction_rack.wells(0, to=counter + 1)])
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),
                     [x.top() for x in reaction_rack_2.wells(0, to=counter + 1)])

    #print(counter)
    robot.home()
amine_transfer(amines_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well D1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A3>',
 'Dispensing 132.0 at <D

In [23]:
from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
amines_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\amines_1_template_corrected.csv")
solvent_df = read_csv(r"C:\Users\opentrons\protocols\PHIP_Feb2019\Solvents.csv")
robot.reset()

def amine_transfer (amines, solvent):
    #Deck setup
    
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "B2")
    tiprack_1000_3 = containers.load("tiprack-1000ul-H", "B1")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_AM_1 = containers.load("FluidX_24_5ml", "A1", "AM_1")
    rack_stock_AM_2 = containers.load("FluidX_24_5ml", "A2", "AM_2")
    rack_stock_AM_3 = containers.load("FluidX_24_5ml", "A3", "AM_3")
    rack_stock_AM_4 = containers.load("FluidX_24_5ml", "B1", "AM_4")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    reaction_rack_2 = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000,tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"    
    solvent = "DMF"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent = "DMF"
    solvent_volume_header = "Volume to dispense (uL)"
    counter =0
    for i, x in enumerate(amines_df[rack_ID_header].tolist()):
        volume_per_reaction = [amines_df[volume_per_reaction_header].tolist()[i]]
        source_location = amines_df[location_header].tolist()[i]
        
        if x == "":
            print ('null')
            break
        if x == rack_1:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_2:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_2.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location),reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_3:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_3.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        if x == rack_4:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_AM_4.wells(source_location), reaction_rack.wells(i+2).top(-5),new_tip='never')
            p1000.transfer(volume_per_reaction, rack_stock_AM_1.wells(source_location), reaction_rack_2.wells(i + 2).top(-5))
        
        counter+=1
    #p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]
    
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc), [x.top() for x in reaction_rack.wells(0, to=counter + 1)])
    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),[x.top() for x in reaction_rack_2.wells(0, to=counter + 1)])
    
    print (counter)
    robot.home()
amine_transfer(amines_df, solvent_df)
robot.commands()

null
58


['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well D1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container AM_1><Well A3>',
 'Dispensing 132.0 at <D